In [1]:
import polars as pl
import pandas as pd
import sqlalchemy
from typing import Dict, List

In [2]:
data = pl.read_csv('lds_test_data.csv')

In [3]:
min_firmID = 0

firm_cols = ["Company Name", "Employee Name"]
firm_df = data.select(firm_cols)
firm_df.with_columns(

)

In [4]:
firm_df

Company Name,Employee Name
str,str
"""McDonalds""","""Joe"""
"""Wendys""","""Darla"""
"""Ad Astra""","""Kevin"""
"""Starbucks""","""Peggy"""


In [5]:
min_surveyNum = 0

questions = [col for col in data.columns if col not in firm_cols]
question_data = {
    "surveyNum": [x for x in range(min_surveyNum, min_surveyNum+len(questions))],
    "responseType": [""]*len(questions),
    "questionText": questions
}

question_df = pl.DataFrame(data=question_data)
question_df.head()

surveyNum,responseType,questionText
i64,str,str
0,"""""","""Number of Lawy…"
1,"""""","""Total Revenue"""
2,"""""","""HQ"""


In [6]:
min_responseID = 0

response_data:Dict[str, List] = {
    "responseID": [x for x in range(min_responseID, question_df.shape[0]*data.shape[0])],
    "firmID": [],
    "surveyNum": [],
    "valueDecimal": [],
    "valueInt": [],
    "valueString": []
}
for question in question_df["questionText"]:
    if data[question].is_utf8():
        for response in data[question]:
            response_data["surveyNum"].append(question_df.filter(question_df["questionText"]==question).select("surveyNum").item())
            
            response_data["valueString"].append(response)
            response_data["valueInt"].append(0)
            response_data["valueDecimal"].append(0.0)

    if data[question].is_numeric():
        for response in data[question]:
            response_data["surveyNum"].append(question_df.filter(question_df["questionText"]==question).select("surveyNum").item())
            response_data["valueString"].append("")
            response_data["valueInt"].append(response)
            response_data["valueDecimal"].append(0.0)

    if data[question].is_float():
        for response in data[question]:
            response_data["surveyNum"].append(question_df.filter(question_df["questionText"]==question).select("surveyNum").item())
            response_data["valueString"].append("")
            response_data["valueInt"].append(0)
            response_data["valueDecimal"].append(response)

In [7]:
response_df = pl.DataFrame(response_data)

In [8]:
response_df

responseID,surveyNum,valueDecimal,valueInt,valueString
i64,i64,f64,i64,str
0,0,0.0,12,""""""
1,0,0.0,14,""""""
2,0,0.0,20,""""""
3,0,0.0,2,""""""
4,1,0.0,200000,""""""
5,1,0.0,100000,""""""
6,1,0.0,50000,""""""
7,1,0.0,10000,""""""
8,2,0.0,0,"""US"""


In [9]:
question_df

surveyNum,responseType,questionText
i64,str,str
0,"""""","""Number of Lawy…"
1,"""""","""Total Revenue"""
2,"""""","""HQ"""


In [26]:
con = sqlalchemy.create_engine("sqlite:///lds_temp.db")
question_df.to_pandas().to_sql("questionIndex", con=con)

ValueError: Table 'questionIndex' already exists.

In [31]:
con = sqlalchemy.create_engine("sqlite:///lds_temp.db")
pd.read_sql_table("questionIndex", con=con, index_col="index")

,surveyNum,responseType,questionText
index,,,
0,0,,Number of Lawyers
1,1,,Total Revenue
2,2,,HQ
